# sscws Conjunction Example Jupyter Notebook

This [Jupyter notebook](https://jupyter.org) demonstrates using the [sscws](https://pypi.org/project/sscws/) Python package to find satellite conjunctions using NASA's [Satellite Situation Center](https://sscweb.gsfc.nasa.gov/) (SSC).  Additionally, it demonstrates using [cdasws](https://pypi.org/project/cdasws) Python package to get data from NASA's [Coordinated Data Analysis System](https://cdaweb.gsfc.nasa.gov) (CDAS) for the satellites and ground stations involved in the conjunction.  This notebook contains the following sections:

1. [Prerequisites](#Prerequisites)
2. [Setup](#Setup)
3. [Define Conjunction Query](#Define-Conjunction-Query)
4. [Run Conjunction Query](#Run-Conjunction-Query)
5. [Display the results](#Display-the-results)
6. [Setup For Access To CDAWeb Data](#Setup-For-Access-To-CDAWeb-Data)
7. [Get Data During Conjunctions](#Get-Data-During-Conjunctions)
8. [Additional Documentation](#Additional-Documentation)

## Prerequisites
For just satellite conjunctions, install the following software from the [Python Package Index](https://pypi.org/project/sscws/) if it is not already installed.

In [1]:
#%pip install sscws

To also get data from cdaweb, install the following additional software from the [Python Package Index](https://pypi.org/).

In [2]:
#%pip install xarray
#%pip install cdflib
#%pip install cdasws

## Setup
Execute some preliminary code that is necessary before the code that follows.

In [3]:
import numpy as np
from sscws.sscws import SscWs
from sscws.conjunctions import BoxConjunctionArea, ConditionOperator,\
    GroundStationCondition, GroundStationConjunction,\
    Satellite, SatelliteCondition, TraceCoordinateSystem
from sscws.coordinates import CoordinateComponent, CoordinateSystem,\
    SurfaceGeographicCoordinates
from sscws.request import DataRequest, QueryRequest, SatelliteSpecification
from sscws.timeinterval import TimeInterval
from sscws.tracing import BFieldTraceDirection, TraceType
ssc = SscWs()

ModuleNotFoundError: No module named 'sscws'

## Define Conjunction Query
The following code defines a query to find magnetic field line conjunctions of at least two THEMIS satellites with one of four THEMIS ground stations during the time from 2008-01-05T10:00:00Z to 2008-01-05T11:59:59Z.

In [ ]:
sats = [
    Satellite('themisa', BFieldTraceDirection.SAME_HEMISPHERE),
    Satellite('themisb', BFieldTraceDirection.SAME_HEMISPHERE),
    Satellite('themisc', BFieldTraceDirection.SAME_HEMISPHERE),
    Satellite('themisd', BFieldTraceDirection.SAME_HEMISPHERE),
    Satellite('themise', BFieldTraceDirection.SAME_HEMISPHERE)
]
satellite_condition = SatelliteCondition(sats, 2)

box_conjunction_area = BoxConjunctionArea(TraceCoordinateSystem.GEO,
                                          3.00, 10.00)

ground_stations = [
    GroundStationConjunction('FSMI', 'THM_Fort Smith',\
        SurfaceGeographicCoordinates(59.98, -111.84),\
        box_conjunction_area),
    GroundStationConjunction('WHIT', 'THM_White Horse',\
        SurfaceGeographicCoordinates(61.01, -135.22),\
        box_conjunction_area),
    GroundStationConjunction('FSIM', 'THM_Fort Simpson',\
        SurfaceGeographicCoordinates(61.80, -121.20),\
        box_conjunction_area),
    GroundStationConjunction('GAK', 'THM_HAARP/Gakona',\
        SurfaceGeographicCoordinates(62.40, -145.20),\
        box_conjunction_area)
]
ground_station_condition = \
    GroundStationCondition(ground_stations,
                            TraceCoordinateSystem.GEO,
                            TraceType.B_FIELD)
conditions = [
    satellite_condition,
    ground_station_condition
]
query_request = \
    QueryRequest('Magnetic conjunction of at least 2 THEMIS satellites with one of 4 THEMIS ground stations during 2008 doy=1-5.',
                 TimeInterval('2008-01-05T10:00:00Z',
                              '2008-01-05T11:59:59Z'),
                 ConditionOperator.ALL,
                 conditions)


## Run Conjunction Query
The following code submits the conjunction query to [SSC](https://sscweb.gsfc.nasa.gov) for evaluation.

In [ ]:
result = ssc.get_conjunctions(query_request)

## Display the results
The following code displays the results of the conjunction query.

In [ ]:
SscWs.print_conjunction_result(result)

---
## Setup For Access To CDAWeb Data
The following is code to find the corresponding data from [cdaweb](https://cdaweb.gsfc.nasa.gov/).  Note that you need to have cdasws and related software installed as descripted in [prerequisites](#Prerequisites) for the following.

In [ ]:
import re
from cdasws.cdasws import CdasWs
from cdasws.datarepresentation import DataRepresentation
import matplotlib.pyplot as plt
cdas = CdasWs()
gs_id = {
    'THM_Fort Smith': 'FSMI',
    'THM_White Horse': 'WHIT',
    'THM_Fort Simpson': 'FSIM',
    'THM_HAARP/Gakona': 'GAK'
}

def get_cdaweb_ds(name: str) -> str:
    if name.startswith("THM"):
        return 'THG_L2_MAG_' + gs_id[name]
    else:
        return 'TH' + name[-1].upper() + '_L2_FGM'

def get_a_mag_var_name(ds: str, names) -> str:
    match = re.search(r'^THG_L2_MAG_(\w*)$', ds)
    if match:
        station = match.group(1)
        return 'thg_mag_' + station.lower()
    else:
        for name in names:
            if name.endswith('fgs_btotal'):
                return name
    return None

## Get Data During Conjunctions
The following code gets data from cdaweb during the conjunctions and plots it.

In [ ]:
for conjunction in result['Conjunction']:
    time_interval = conjunction['TimeInterval']
    start = time_interval['Start']
    end = time_interval['End']
    sats = []
    for sat_des in conjunction['SatelliteDescription']:
        sats.append(sat_des['Satellite'])
    sat = conjunction['SatelliteDescription'][0]['Satellite']
    gs = conjunction['SatelliteDescription'][0]['Description'][0]['TraceDescription']['Target']['GroundStation']
    datasets = [get_cdaweb_ds(gs)]
    for sat in sats:
        datasets.append(get_cdaweb_ds(sat))
    got_ds_data = False
    for ds in datasets:
        var_name = get_a_mag_var_name(ds, cdas.get_variable_names(ds))
        #print(f'requesting: {ds}, {start} to {end}, {var_name}')
        status, data = cdas.get_data(ds, [var_name], start, end,
                                     dataRepresentation = DataRepresentation.XARRAY)
        if status['http']['status_code'] != 200:
            print(f'Failed to get {ds} {var_name} {start} to {end} data from CDAWeb')
            print(f'status: {status}')
            continue
        got_ds_data = True
        var_data = data[var_name]
        epoch_var_name = var_data.attrs['DEPEND_0']
        if var_data.ndim == 2:
            var_data = np.linalg.norm(var_data, axis = 1)
            data_label = 'delta magnitude ' + var_name
        else:
            data_label = 'delta ' + var_name
        mean = var_data.mean(axis = 0)
        delta_data = var_data - mean
        plt.plot(data[epoch_var_name], delta_data, label = data_label)

    if got_ds_data:
        plt.xlabel('UTC: ' + start.date().isoformat())
        plt.ylabel('Delta Magnetic Field Magnitude (nT)')
        plt.legend()
        plt.show()

## Additional Documentation
View the [sscws API](../py/sscws/) for additional features.  Additional [notebook examples](../#Jupyter_Notebook_Examples) are also available.